1\. Write a function that converts number representation (bin<->dec<->hex)

In [6]:
def converter(n, repres):
    if repres == "dec":
        return int(n)
    elif repres == "bin":
        return bin(n)
    elif repres == "hex":
        return hex(n)
    
x = 145
y = 0x145
z = 0b10101010
print(converter(x,"bin"))
print(converter(x,"hex"))
print(converter(y,"dec"))
print(converter(y,"bin"))
print(converter(z,"dec"))
print(converter(z,"hex"))

0b10010001
0x91
325
0b101000101
170
0xaa


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [53]:
import numpy as np

def singleprec(w):
    masksign = 0b10000000000000000000000000000000
    maskexp =  0b01111111100000000000000000000000
    maskmant = 0b00000000011111111111111111111111
    sign = (w & masksign) >> 31
    exp = (w & maskexp) >> 23
    m = (w & maskmant)
    mant = 1
    n = 23
    for i in range(1,24):
        mant = mant + (((m & (2**(n-i))) >> (n-i))*(2**(-i))) #calculate mantissa
    result = ((-1)**sign)*mant*(np.float_power(2,exp-127))
    return result

n1 = np.uint32(65011712)
n2 = np.uint32(3232759808)
print(bin(n1))
print(bin(n2))
r1 = singleprec(n1)
print(r1)
r2 = singleprec(n2)
print(r2)

0b11111000000000000000000000
0b11000000101100000000000000000000
1.316553672920962e-36
-5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [2]:
x = 1
y = 1
xx = 1
yy = 1
i = 0

while x != 0:
    xx = x
    x = x/2
print("Underflow limits is:",xx)

print("Can't find an overflow number in feasible time. Python3 represent integers with arbitrary precision")



Underflow limits is: 5e-324
Can't find an overflow number in feasible time. Python3 represent integers with arbitrary precision


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [34]:
i = 1
n,nn = 1,1
while i<1000:
    n = nn + (10**(-i))
    i += 1
    if nn == n:
        break
    
print(i)

17


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [25]:
import math
a,b,c = 0.001,1000,0.001

P = math.sqrt((b**2)-(4*a*c))

# Point A
x1 = ((-b)+P)/(2*a)
x2 = ((-b)-P)/(2*a)

print(x1,x2)

print(a*(x1**2)+b*x1+c)
print(a*(x2**2)+b*x2+c)

# Point B
x11 = (((-b)+P)*((-b)+P))/((2*a)*((-b)+(P)))
x12 = (((-b)-P)*((-b)+P))/((2*a)*((-b)+(P)))
x21 = (((-b)+P)*((-b)-P))/((2*a)*((-b)-(P)))
x22 = (((-b)-P)*((-b)-P))/((2*a)*((-b)-(P)))

print(x11,x12)
print(x21,x22)

# Point C
def quadeq(a,b,c):
    x1 = (2*c)/((-b)-P)
    x2 = ((-b)-P)/(2*a)
    return x1,x2
x3 = x1
x4 = x2
x1,x2 = quadeq(a,b,c)

print(x3-x1, x4-x2)
print(x1,x2)
print(a*(x1**2)+b*x1+c)
print(a*(x2**2)+b*x2+c)

-9.999894245993346e-07 -999999.999999
1.0575401665491313e-08
7.247924804689582e-08
-9.999894245993346e-07 -999999.9999990001
-9.999894245993346e-07 -999999.999999
1.0575401665491313e-11 0.0
-1.000000000001e-06 -999999.999999
0.0
7.247924804689582e-08


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [14]:
def f(x):
    return x*(x-1)

x = 1
delta = [10**(-i) for i in range(1,15) if i%2 == 0]

realderiv = 2*x-1
diff = []

for j in delta:
    deriv = (f(x+j)-f(x))/j
    diff.append(realderiv-deriv)

print(diff)

[-0.010000000000000897, -9.999999988985486e-05, -9.99917733279787e-07, -3.922528746258536e-09, -8.284037100736441e-08, -8.890058334132256e-05, 0.0007992778373491216]


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [64]:
import math
import time

def riemann(N):
    h = 2/N
    I2 = 0
    for i in range(N):
        I2 += (1-(-1+(h*i))**2)**(0.5)
    return h*I2

pi = math.pi
I = pi/2

N = 100
st = time.time()
I2 = riemann(N)
end = time.time()-st

print("N =",N)
print("Time to compute",end)
print("Value:",I2)
print("Difference with the true value",I-I2)

# less than a second
N = 4150000
st = time.time()
I2 = riemann(N)
end = time.time()-st

print("\nN = 4150000")
print("Time to compute",end)
print(I2)
print("Difference with the true value",I-I2)

# less than a minute
N = 250000000
st = time.time()
I2 = riemann(N)
end = time.time()-st

print("\nN =",N)
print("Time to compute",end)
print("Value:",I2)
print("Difference with the true value",I-I2)


N = 100
Time to compute 0.0004317760467529297
Value: 1.5691342555492498
Difference with the true value 0.001662071245646768

N = 4150000
Time to compute 0.9938905239105225
1.5707963265981584
Difference with the true value 1.967381813017255e-10

N = 250000000
Time to compute 56.716684341430664
Value: 1.5707963267945662
Difference with the true value 3.304023721284466e-13
